In [42]:
#김도윤_12/17_수정본
#주단위 데이터 -> 월단위 데이터 수정
#평균데이터 추가
#가시성을 위해 평균데이터에 강조효과 추가
#탭 하나로 통일
#밑에 대사 추가
#이승용 화이팅
#SSG 랜더스 화이팅
#이정후 화이팅
#안타 안타 날려버려라 키움 히어로 이정후



# 라이브러리 임포트
import pandas as pd
import folium
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px  # Import plotly express

# Folium 지도 위경도 데이터
file = pd.read_csv('yeo_pm.csv', encoding='cp949')
station = pd.DataFrame(file)

# Folium 지도
initial_location = [37.525989, 126.923948]
my_map = folium.Map(location=initial_location, zoom_start=14)
for index, row in station.iterrows():
    lat, lon = row['X'], row['Y']
    folium.Marker(location=[lat, lon], popup=row['대여소명']).add_to(my_map)

# 파일 데이터 불러오기(따릉이 시간대별 대여량)
rental_data = {
    f'{month}월 시간대별 대여현황': {f'OD/tpss_bcycl_od_statnhm_2023{month:02d}{day:02d}.csv': f'{month:02d}/{day:02d}' for day in range(1, 32) if day <= (30 if month in {4, 6} else 28 if month == 2 else 31)}
    for month in range(1, 7)
}

# 파일 데이터 불러오기(따릉이 자치구별 대여량)
file_paths = {
    '1월 대여소별 대여현황': 'PM_CNT_1.csv',
    '2월 대여소별 대여현황': 'PM_CNT_2.csv',
    '3월 대여소별 대여현황': 'PM_CNT_3.csv',
    '4월 대여소별 대여현황': 'PM_CNT_4.csv',
    '5월 대여소별 대여현황': 'PM_CNT_5.csv',
    '6월 대여소별 대여현황': 'PM_CNT_6.csv'
}

In [43]:
# 대쉬 및 레이아웃 구성
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("여의도 내 따릉이 대여소 및 대여현황"),
    #  여의도내 따릉이 대여소 위치현황
    html.Div([
        html.Iframe(id='map', srcDoc=my_map._repr_html_(), width='100%', height='600'),
        html.P("주제: 여의도 내 지하철역과 서울시 공공자전거 정류소 간 상관성 및 환승체계 도입의 필요성"
               "만약 서울시 내 대중교통 환승시스템에 따릉이가 포함된다면 이용률에 영향을 미칠까? 라는 의문에서 조사를 시작했다."
               "조사대상지: 서울특별시 영등포구 여의도 일대\n"
               "조사 대상지인 여의도 일대와 따릉이 대여소 현황이다.\n"
               "현재 43개의 정류소가 여의도 내 위치하고 있다."
               "다음은 대상지 선정과정이다."),
    ]),

    # 따릉이 시간대별 대여현황
    html.Div([
        html.H2("따릉이 시간대별 대여현황"),
        dcc.Dropdown(
            id='file-dropdown',
            options=[
                {'label': key, 'value': key} for key in rental_data.keys()
            ],
            value='1월 시간대별 대여현황',
            style={'width': '50%'}
        ),
        dcc.RadioItems(
            id='weekday-weekend-radio',
            options=[
                {'label': '평일', 'value': '평일'},
                {'label': '주말', 'value': '주말'}
            ],
            value='평일', 
            style={'margin-top': '10px'}
        ),
        dcc.Graph(id='line-plot'),
        html.P("서울열린데이터 광장에서 수집한 2023년 1월1일부터 6월30일까지의 시간대별 따릉이 이용 데이터다."
               "출퇴근 시간대에 높은 첨두교통량을 보인다. 따라서 출퇴근 용도로 따릉이를 많이 사용한다는 가설이 도출된다."
               "주말과 평일을 비교한다면 평일에 이용량이 더 많은 모습을 보인다."
               "추가로 출근시간대에 급격히 이용량이 증가하고 감소하는 모습이 보인다."
               "그러나 퇴근시간대에 급격한 이용량 증가 후 감소폭은 출근시간대에 비해 완만하다."
               "비가 오거나 미세먼지 농도가 높은 날은(ex: 04/05, 05/06) 이용량이 대폭 감소한다."
               "1월에서 6월까지 날씨가 온난해짐에 따라 전체적인 이용량이 증가한다."
               "위 사례를 통해 자전거 이동수단의 특성과 이용용도를 파악할 수 있다."
               "이와 같은 조사를 통해 조사 대상지를 출퇴근이 많이 발생하는 곳으로 후보군을 선정했다."
               "서울시내 중심업무지구 '종로, 강남, 여의도'를 선정했다."),
    ]),

    # 따릉이 자치구별 대여현황
    html.Div([
        html.H2("따릉이 자치구별 대여현황"),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': month, 'value': month} for month in file_paths.keys()],
            value='1월 대여소별 대여현황',
            style={'width': '50%'}
        ),
        dcc.Graph(id='bar-chart'),
        html.P("서울열린데이터 광장에서 수집한 2023년 1월부터 6월까지 자치구별 따릉이 대여현황이다."
               "위 데이터에 따르면 강서구, 송파구, 영등포구 순으로 이용량이 많음을 알 수 있다."
               "따라서 두 조건을 모두 만족하는 영등포구 여의도 일대를 조사 대상지로 선정했다."
               "여의도의 경우 따릉이를 가장 먼저 도입한 시범지역이며 한강으로 둘러싸여있어 조사대상지를 명확히 구분할 수 있다."),
    ]),
])

In [44]:
#콜백구성
# 시간대별 대여현황 콜백
@app.callback(
    Output('line-plot', 'figure'),
    [Input('file-dropdown', 'value'),
     Input('weekday-weekend-radio', 'value')]
)
def update_line_plot(selected_mapping, weekday_weekend):
    combined_data = pd.DataFrame()

    for filename, legend_label in rental_data[selected_mapping].items():
        data = pd.read_csv(filename, encoding='cp949')
        
        # '기준_날짜' 열을 날짜 형식으로 변환
        data['기준_날짜'] = pd.to_datetime(data['기준_날짜'], format='%Y%m%d')
        
        # 선택한 옵션에 따라 주중 또는 주말을 필터링
        if weekday_weekend == '평일':
            data = data[data['기준_날짜'].dt.weekday < 5]  # 월요일부터 금요일까지가 주중 (0부터 4)
        elif weekday_weekend == '주말':
            data = data[data['기준_날짜'].dt.weekday >= 5]  # 토요일과 일요일이 주말 (5와 6)

        rental_count_by_hour = data.groupby('기준_시간대').size()
        combined_data[legend_label] = rental_count_by_hour

    # Calculate the overall average
    combined_data['평균 대여량'] = combined_data.mean(axis=1)

    fig = px.line(combined_data, x=combined_data.index, y=combined_data.columns, markers=True)

    # '평균 대여량'의 선과 점 굵기와 색상 조절
    fig.update_traces(selector=dict(name='평균 대여량'), line=dict(width=5, color='black'), marker=dict(size=7))

    fig.update_layout(
        title=f'{weekday_weekend} 시간대별 따릉이 대여현황',
        xaxis_title='시간대',
        yaxis_title='대여 횟수',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

    return fig

#자치구별 대여현황 콜백
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('month-dropdown', 'value')]
)
def update_chart(selected_month):
    data = pd.read_csv(file_paths[selected_month], encoding='cp949')
    
    # Group by '자치구' and sum '이용합계'
    grouped_data = data.groupby('자치구')['이용합계'].sum().reset_index()


    fig = px.bar(grouped_data, x='자치구', y='이용합계', title=f'따릉이 {selected_month}', labels={'이용합계': '이용 합계'})
    return fig

In [45]:
if __name__ == '__main__':
    app.run_server(debug=True, port = 1220)